In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd

In [ ]:
class RLEnv():
    def initialize(self, Path, PortfolioValue = 10000, TransCost = 0.0025, ReturnRate = 0.02/250, 
                   WindowSize = 50, TrainTestSplit = 0.7):
        self.Dataset = np.load(Path)
        self.NumStocks = self.Dataset.shape[1]
        self.NumValues = self.Dataset.shape[0]
        self.PortfolioValue = PortfolioValue
        self.TransCost = TransCost
        self.ReturnRate = ReturnRate
        self.WindowSize = WindowSize
        self.Done = False
        self.state = None
        self.TimeLength = None
        self.Terminate = False
        self.TerminateRows = int((self.Dataset.shape[2] - self.WindowSize) * TrainTestSplit)
        
    def UpdatedOpenValues(self, T):
        return np.array([1+self.ReturnRate]+self.Dataset[-1,:,T].tolist())
    
    def InputTensor(self, Tensor, T):
        return Tensor[: , : , T - self.WindowSize:T]
    
    def ResetEnvironment(self, InitWeight, InitPortfolio, T):
        self.state= (self.InputTensor(self.Dataset, self.WindowSize) , InitWeight , InitPortfolio)
        self.TimeLength = self.WindowSize + T
        self.Done = False
        
        return self.state, self.Done
    
    def Step(self, Action):

        Dataset = self.InputTensor(self.Dataset, self.index)
        weight_vector_old = self.state[1]
        portfolio_value_old = self.state[2]
        NewOpenValues = self.UpdatedOpenValues(index)
        WeightAllocation = Action
        PortfolioAllocation = portfolio_value_old
        TransactionCost = PortfolioAllocation * self.TransCost * np.linalg.norm((WeightAllocation-weight_vector_old),ord = 1)
        ValueAfterTransaction = (PortfolioAllocation * WeightAllocation) - np.array([cost]+ [0]*self.nb_stocks)
        NewValueofStocks = ValueAfterTransaction * NewOpenValues
        NewPortfolioValue = np.sum(NewValueofStocks)
        NewWeightVector = NewValueofStocks/NewPortfolioValue
        RewardValue = (NewPortfolioValue - portfolio_value_old)/(portfolio_value_old)

        self.index = self.index + 1
        self.state = (self.InputTensor(self.Dataset, self.index), NewWeightVector, NewPortfolioValue)
        if index >= self.TerminateRows:
            self.Done = True
        return self.state, self.RewardValue, self.Done  